<a href="https://colab.research.google.com/github/sandrazuniga/Challenge_TelecomX_Parte2/blob/main/TelecomX_Parte2_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telecom X - Análisis de Evasión de Clientes

La empresa enfrenta una alta tasa de cancelaciones y necesita comprender los factores que llevan a la pérdida de clientes.

El ánalisis contiene:

Importación y manipulación de datos desde una API.
Aplicación de los conceptos de ETL (Extracción, Transformación y Carga) en la preparación de los datos.
Creación de visualizaciones estratégicas para identificar patrones y tendencias.
Análisis Exploratorio de Datos (EDA) e informe con insights relevantes.

#📌 Extracción

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from matplotlib.lines import Line2D
import seaborn as sns
import requests
import json

In [ ]:
url = 'https://raw.githubusercontent.com/sandrazuniga/Challenge_TelecomX_Parte2/refs/heads/main/TelecomX_Data.json'

df = pd.read_json(url)

In [ ]:
df.head()

In [ ]:
df.info()

#🔧 Transformación

In [ ]:
# Paso 1: Normalizar columnas anidadas
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

# Paso 2: Eliminar las columnas anidadas originales del DataFrame
df.drop(columns=['customer', 'phone', 'internet', 'account'], inplace=True)

# Paso 3: Concatenar el DataFrame original con los nuevos datos normalizados
df = pd.concat([df, customer_df, phone_df, internet_df, account_df], axis=1)

# Paso 4: Verificar los cambios
df.head()

In [ ]:
df.columns

In [ ]:

df.rename(columns={
    'customerID': 'ID',
    'Churn':'cancelo',
    'gender': 'genero',
    'SeniorCitizen': 'mayor_de_65',
    'Partner': 'tiene_pareja',
    'Dependents': 'tiene_dependentes',
    'tenure': 'meses_de_contrato',
    'PhoneService': 'servicio_telefonico',
    'MultipleLines': 'lineas_multiples',
    'InternetService': 'servicio_internet',
    'OnlineSecurity': 'seguridad_en_linea',
    'OnlineBackup': 'Soporte_en_linea',
    'DeviceProtection': 'proteccion_dispositivos',
    'TechSupport': 'soporte_tecnico',
    'StreamingTV': 'servicio_tv',
    'StreamingMovies': 'servicio_peliculas',
    'PaperlessBilling': 'facturas_electronicas',
    'PaymentMethod': 'metodo_pago',
    'Contract': 'tipo_contrato',
    'Charges.Monthly': 'gastos_mensuales',
    'Charges.Total': 'gastos_totales'
}, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
(df['cancelo'] == '').sum()

In [ ]:
(df['gastos_totales'] == ' ').sum()

In [ ]:
df['cancelo'] = df['cancelo'].replace('', np.nan)
df['gastos_totales'] = df['gastos_totales'].replace(' ', np.nan)

In [ ]:
(df['cancelo'] == '').sum()
(df['gastos_totales'] == ' ').sum()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(subset=['cancelo', 'gastos_totales'])

In [ ]:
df[['cancelo', 'gastos_totales']].isnull().sum()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Convertir a float
df['gastos_totales'] = df['gastos_totales'].astype(np.float64)

In [ ]:
# Convertir a str
vars_a_str = ['ID', 'genero', 'servicio_internet', 'tipo_contrato', 'metodo_pago']

In [ ]:
# Convertirlas a string
df[vars_a_str] = df[vars_a_str].astype(str)

In [ ]:
# creo una variable que contiene todas las variables que van a ser transformadas en booleanos

columnas_bool = ['cancelo','mayor_de_65', 'tiene_pareja', 'tiene_dependentes', 'facturas_electronicas']
df[columnas_bool] = df[columnas_bool].replace({'Yes': True, 'No': False}).infer_objects(copy=False)

In [ ]:
# Transformación de variables a categoricas

columnas_categoricas = ['lineas_multiples', 'seguridad_en_linea', 'Soporte_en_linea', 'proteccion_dispositivos', 'soporte_tecnico', 'servicio_tv', 'servicio_peliculas']
df[columnas_categoricas] = df[columnas_categoricas].astype('category')

In [ ]:
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df['cuentas_diarias'] = df['gastos_mensuales'].astype(float) / 30

In [ ]:
df.head()

In [ ]:
df[columnas_bool] = df[columnas_bool].astype(int)

In [ ]:
df[columnas_bool].head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.dtypes

#📊 Carga y análisis

In [ ]:
df.describe()

In [ ]:
# identifica variables categoricas

columnas_categoricas = df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
columnas_categoricas

In [ ]:
# Realizar recuentos cruzados con cancelo

for col in columnas_categoricas:
    print(f'\n--- Porcentaje de evasión por {col} ---')
    porcentaje = pd.crosstab(df[col], df['cancelo'], normalize='index') * 100
    print(porcentaje.round(2))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Definimos los colores
AZUL = '#1f77b4'         # azul corporativo
VERDE = '#2ca02c'        # verde corporativo
GRIS = '#4D4D4D'         # gris para títulos

In [ ]:
def cancelacion_tipo_internet():

  # Ordenar categorías según porcentaje de cancelación
  orden = df.groupby('servicio_internet')['cancelo'].mean().sort_values().index

  # Crear gráfica
  plt.figure(figsize=(10, 6))
  ax = sns.barplot(
      x='servicio_internet',
      y='cancelo',
      data=df,
      estimator=lambda x: x.mean() * 100,
      order=orden,
      errorbar=None
  )

  # Asignar colores barra por barra (sin palette -> sin warning)
  for i, bar in enumerate(ax.patches):
      color = AZUL if i % 2 == 0 else VERDE
      bar.set_facecolor(color)

  # Añadir porcentaje sobre cada barra
  for container in ax.containers:
      ax.bar_label(container, fmt='%.1f%%', label_type='edge', padding=3, color=AZUL, fontsize=12)

  # Personalización
  ax.set_title('Cancelación por tipo de Internet', fontsize=18, color=GRIS)

  # Eje Y: suprimir ticks y etiqueta
  ax.set_ylabel('')
  ax.set_yticklabels([])  # oculta los valores del eje y
  ax.tick_params(axis='y', length=0)  # oculta las marcas de ticks del eje y

  # Eje X: etiquetas en azul, tamaño 12
  ax.set_xlabel('')
  ax.tick_params(axis='x', labelsize=12, colors=AZUL)

  # Eliminar bordes superior, derecho e izquierdo
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)

  plt.tight_layout()



  return plt

In [ ]:
def cancelacion_tipo_contrato():

  # Ordenar categorías según porcentaje de cancelación
  orden = df.groupby('tipo_contrato')['cancelo'].mean().sort_values().index

  # Crear gráfica
  plt.figure(figsize=(10, 6))
  ax = sns.barplot(
      x='tipo_contrato',
      y='cancelo',
      data=df,
      estimator=lambda x: x.mean() * 100,
      order=orden,
      errorbar=None
  )

  # Asignar colores barra por barra (alternancia azul/verde)
  for i, bar in enumerate(ax.patches):
      color = AZUL if i % 2 == 0 else VERDE
      bar.set_facecolor(color)

  # Añadir porcentaje sobre cada barra
  for container in ax.containers:
      ax.bar_label(container, fmt='%.1f%%', label_type='edge', padding=3, color=AZUL, fontsize=12)

  # Personalización
  ax.set_title('Cancelación por tipo de contrato', fontsize=18, color=GRIS)

  # Eje Y: suprimir ticks y etiqueta
  ax.set_ylabel('')
  ax.set_yticklabels([])  # oculta los valores del eje y
  ax.tick_params(axis='y', length=0)  # oculta las marcas de ticks del eje y

  # Eje X: etiquetas en azul, tamaño 12
  ax.set_xlabel('')
  ax.tick_params(axis='x', labelsize=12, colors=AZUL)

  # Eliminar bordes superior, derecho e izquierdo
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)

  # Fondo gris
  ax.set_facecolor('#f0f0f0')  # Fondo del gráfico
  plt.gcf().set_facecolor('#f0f0f0')  # Fondo de la figura completa

  plt.tight_layout()

  return plt


def cancelacion_servicios_adicionales():

  # Lista de servicios adicionales
  servicios = ['seguridad_en_linea', 'Soporte_en_linea', 'proteccion_dispositivos', 'soporte_tecnico']
  titulos = ['Seguridad en línea', 'Soporte en línea', 'Protección de dispositivos', 'Soporte técnico']

  # Orden lógico de categorías
  orden = ['No', 'Yes', 'No internet service']

  # Crear figura y ejes
  fig, axs = plt.subplots(2, 2, figsize=(12, 8))
  fig.subplots_adjust(hspace=0.5, wspace=0.3)
  fig.suptitle('Cancelación según servicios adicionales', fontsize=20, color=GRIS)
  fig.patch.set_facecolor('#f0f0f0')  # fondo de la figura

  # Gráficas
  for i, (servicio, titulo) in enumerate(zip(servicios, titulos)):
      fila = i // 2
      col = i % 2
      ax = axs[fila, col]
      ax.set_facecolor('#f0f0f0')  # fondo del gráfico individual

      # Gráfica de barras sin usar hue
      plot = sns.barplot(
      x=servicio,
      y='cancelo',
      data=df,
      estimator=lambda x: x.mean() * 100,
      order=orden,
      errorbar=None,
      hue=servicio,           # agregamos hue (aunque no usaremos leyenda)
      legend=False,           # ocultamos leyenda
      ax=ax,
      palette=[AZUL, VERDE, AZUL]
  )


      # Título
      ax.set_title(titulo, fontsize=16, color=GRIS)

      # Etiqueta del eje Y solo en la primera columna
      if col == 0:
          ax.set_ylabel('')
          ax.set_yticklabels([])  # suprimir etiquetas y
          ax.tick_params(axis='y', length=0)  # sin ticks eje y
      else:
          ax.set_ylabel('')
          ax.set_yticklabels([])
          ax.tick_params(axis='y', length=0)

      # Eje X
      ax.set_xlabel('')
      ax.tick_params(axis='x', labelsize=12, colors=AZUL)

      # Bordes
      ax.spines['top'].set_visible(False)
      ax.spines['right'].set_visible(False)
      ax.spines['left'].set_visible(False)

      # Rango común para comparar
      ax.set_ylim(0, 50)

      # Etiquetas sobre las barras
      for container in ax.containers:
          ax.bar_label(container, fmt='%.1f%%', label_type='edge', padding=3, color=AZUL, fontsize=12)

  plt.tight_layout(rect=[0, 0, 1, 0.95])  # para que el título no se encime

  return plt

In [ ]:
# Realizar recuentos cruzados con cancelo
variables_binarias = ['mayor_de_65', 'tiene_pareja', 'tiene_dependentes']

for col in variables_binarias:
    print(f'\n--- Porcentaje de evasión por {col} ---')
    porcentaje = pd.crosstab(df[col], df['cancelo'], normalize='index') * 100
    print(porcentaje.round(2))

In [ ]:
def cancelacion_bin():

  plt.figure(figsize=(14, 4))


  for i, var in enumerate(variables_binarias):
      plt.subplot(1, 4, i+1)

      # Calcular porcentaje de cancelación
      data = df.groupby(var)['cancelo'].mean() * 100
      categorias = data.index
      valores = data.values

      # Convertimos a DataFrame para usar x=hue
      temp_df = pd.DataFrame({var: categorias, 'cancelacion': valores})

      # Gráfico de barras con colores alternos
      sns.barplot(
          data=temp_df,
          x=var,
          y='cancelacion',
          hue=var,
          palette=[AZUL, VERDE],
          legend=False
      )

      # Texto sobre las barras
      for j, v in enumerate(valores):
          plt.text(j, v + 2, f'{v:.1f}%', ha='center', fontsize=12, color=AZUL)

      # Limites y personalización
      plt.ylim(0, 100)
      plt.title(var.replace('_', ' ').capitalize(), fontsize=16, )

      # Eje Y sin etiquetas ni ylabel
      plt.tick_params(axis='y', labelleft=False)
      plt.ylabel('')

      # Eje X personalizado
      plt.xlabel('')
      plt.xticks([0, 1], ['No', 'Sí'], fontsize=12, color=AZUL)

      # Eliminar bordes superior, derecho e izquierdo
      ax = plt.gca()
      ax.spines['top'].set_visible(False)
      ax.spines['right'].set_visible(False)
      ax.spines['left'].set_visible(False)

  plt.tight_layout()

  return plt


In [ ]:
# Variables numericas

columnas_numericas = [['gastos_mensuales','gastos_totales','cuentas_diarias']]

# Promedios y desviaciones por grupo
df.groupby('cancelo')[['gastos_mensuales', 'gastos_totales', 'cuentas_diarias']].agg(['mean', 'std'])


In [ ]:
print(columnas_numericas)
print(type(columnas_numericas))

In [ ]:
columnas_numericas = columnas_numericas[0]

In [ ]:
def boxplot_cancelacion(columnas_numericas):

  plt.figure(figsize=(15, 5))
  plt.gcf().set_facecolor('#F2F2F2')  # Fondo claro

  for i, col in enumerate(columnas_numericas):
      plt.subplot(1, 3, i + 1)

      # Boxplot con alternancia de colores
      sns.boxplot(
      x='cancelo',
      y=col,
      hue='cancelo',
      data=df,
      showfliers=False,
      palette=[AZUL, VERDE],
      legend=False
     )



      # Título en GRIS
      plt.title(f'{col.replace("_", " ").capitalize()} según cancelación', fontsize=16, color=GRIS)

      # Eje X
      plt.xlabel('Canceló', fontsize=12, color=AZUL)
      plt.xticks([0, 1], ['No', 'Sí'], fontsize=12, color=AZUL)

      # Eje Y
      plt.ylabel(col.replace('_', ' ').capitalize(), fontsize=12, color=AZUL)
      plt.tick_params(axis='y', labelsize=12, colors=AZUL)

      # Eliminar bordes superior, derecho e izquierdo
      ax = plt.gca()
      ax.set_facecolor('#F2F2F2')  # Fondo claro en subplot
      ax.spines['top'].set_visible(False)
      ax.spines['right'].set_visible(False)
      ax.spines['left'].set_visible(False)

  plt.tight_layout()
  return plt


In [ ]:
def cancelacion_segun_meses():

  # Boxplot para meses de contrato
  plt.figure(figsize=(10, 5))
  sns.boxplot(x='cancelo', y='meses_de_contrato', data=df, showfliers=False)
  plt.title('Meses de contrato según cancelación', fontsize=18, color=GRIS)
  plt.xlabel('Canceló')
  plt.ylabel('Meses de contrato')

  # Eliminar bordes superior, derecho e izquierdo
  ax = plt.gca()
  ax.set_facecolor('#F2F2F2')  # Fondo claro en subplot
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['left'].set_visible(False)


  plt.tight_layout()
  return plt

#📄Informe final

In [ ]:
cancelacion_tipo_internet().show()

In [ ]:
 cancelacion_tipo_contrato().show()

In [ ]:
cancelacion_servicios_adicionales().show()

In [ ]:
cancelacion_bin().show()

In [ ]:
boxplot_cancelacion(columnas_numericas).show()

In [ ]:
cancelacion_segun_meses().show()

🔹 Conclusiones e Insights:
🧪 Principales hallazgos

Mayor cancelación con fibra óptica: Los usuarios con internet de fibra óptica presentan un porcentaje de cancelación más alto (41.9%).

Contratos mes a mes tienen mayor evasión: Las personas con contratos mensuales tienen una tasa de cancelación elevada (42.7%), en comparación con otros tipos de contrato.

Menor fidelización sin servicios adicionales: Los clientes que no cuentan con servicios adicionales cancelan con mayor frecuencia:

Seguridad en línea: 41.8%

Soporte en línea: 39.9%

Protección de dispositivos: 39.1%

Soporte técnico: 41.6%

Edad avanzada y mayor tasa de cancelación: Los clientes mayores de 65 años tienen una tasa de cancelación mucho más alta (41.7%) que los más jóvenes.

Estado civil influye: Los clientes sin pareja cancelan más (~33%) que aquellos con pareja.

Menos dependientes, más cancelación: Los clientes sin personas a su cargo cancelan en mayor proporción (31.3%) que los que sí tienen dependientes.

Mayor gasto mensual, mayor probabilidad de cancelación: A medida que aumenta el gasto mensual, también lo hace la tasa de cancelación. Esto puede deberse a la percepción de que el servicio es costoso.

Clientes nuevos cancelan más: Quienes cancelaron tienen un total facturado significativamente menor, lo que indica que llevaban poco tiempo como clientes.

Más uso no garantiza fidelidad: Los clientes que cancelaron presentan ligeramente más conexiones diarias. Aunque la diferencia no es muy marcada, podría sugerir que usaban el servicio activamente pero no estaban satisfechos.

Contratos cortos = más cancelaciones: Los clientes que cancelaron tienden a tener contratos mucho más cortos (mediana ~10 meses), lo que sugiere menor compromiso o una etapa temprana de prueba del servicio.

🛠️ ¿Cómo reducir la evasión?

Revisar el servicio de fibra óptica: Identificar y corregir posibles fallas técnicas. Ofrecer alternativas tecnológicas (como otro tipo de conexión) en zonas problemáticas para evitar cancelaciones.

Incentivar contratos más duraderos: Ofrecer beneficios como descuentos o promociones para quienes eligen contratos más largos, sin imponer cláusulas de permanencia rígidas.

Incluir servicios adicionales sin recargo: Crear paquetes integrales que incluyan servicios como soporte técnico, seguridad en línea y protección de dispositivos, mejorando así la experiencia del cliente sin aumentar su factura.

Enfocarse en el público mayor: Diseñar ofertas específicas para adultos mayores, con contenidos o canales afines a sus intereses (por ejemplo, películas clásicas o programación religiosa).

Promociones para personas solas o sin dependientes: Brindar beneficios especiales como descuentos o servicios gratuitos adicionales a este grupo, que muestra mayor tendencia a cancelar.

Reducir la percepción de altos costos: Ajustar tarifas o implementar opciones de pago flexibles, como pagos quincenales, para mejorar la percepción de accesibilidad y evitar que los usuarios sientan que están pagando de más.

🔹 Recomendaciones:
Con base en los hallazgos obtenidos, se proponen las siguientes estrategias para reducir la tasa de cancelación y aumentar la fidelización de clientes:

Optimizar la experiencia de nuevos clientes:
Desarrollar un programa de bienvenida que acompañe a los nuevos usuarios durante los primeros meses.

Realizar encuestas de satisfacción temprana para detectar problemas a tiempo y evitar cancelaciones tempranas.

Mejorar la percepción de valor del servicio:
Reestructurar los paquetes para incluir servicios adicionales sin aumentar el precio final.

Comunicar de forma más clara los beneficios incluidos en el servicio (soporte, seguridad, protección).

Implementar opciones de pago más flexibles (ej. pagos quincenales o personalizados) para clientes con alto gasto mensual.

Fidelizar a los clientes con contratos mes a mes:
Ofrecer incentivos exclusivos para quienes migren de contrato mensual a uno de mayor duración (como beneficios permanentes o tarifas especiales).

Crear un sistema de recompensas o puntos por antigüedad, aplicable incluso en contratos mensuales.

Segmentar ofertas según el perfil del cliente:
Para adultos mayores: ofrecer contenido personalizado, soporte preferencial, atención al cliente dedicada y capacitación básica para el uso del servicio.

Para personas solteras o sin dependientes: crear campañas enfocadas en independencia, flexibilidad y autosuficiencia, con promociones diseñadas para su estilo de vida.

Auditar y mejorar el servicio de fibra óptica:
Analizar reclamos técnicos y evaluar la infraestructura por zonas geográficas.

Priorizar mejoras en las áreas con mayores tasas de cancelación asociadas a fibra óptica.

Capacitar al personal técnico para garantizar instalaciones sin inconvenientes.

Monitorear la actividad del usuario:
Identificar patrones de uso que preceden a la cancelación (como reducción en conexiones diarias).

Usar esta información para activar alertas preventivas o campañas personalizadas de retención.

Crear una unidad de análisis de cancelaciones:
Implementar un sistema continuo de análisis de evasión para identificar nuevas tendencias.

Hacer seguimiento mensual de métricas clave (edad, tipo de contrato, servicios adicionales, antigüedad, etc.).

In [ ]:
df.columns

In [ ]:
df

In [ ]:
X = df.drop('cancelo', axis = 1)
y = df['cancelo']

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify = y, random_state=42)

In [ ]:
columnas_categoricas = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
columnas_numericas = ['meses_de_contrato', 'gastos_mensuales']

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
one_hot = make_column_transformer((OneHotEncoder(sparse_output=False, handle_unknown='ignore'), columnas_categoricas), (StandardScaler(), columnas_numericas), remainder='passthrough', sparse_threshold=0)

In [ ]:
X_train_encoded = one_hot.fit_transform(X_train)
X_test_encoded = one_hot.transform(X_test)

In [ ]:
one_hot.get_feature_names_out()

In [ ]:
X_train_df = pd.DataFrame(X_train_encoded, columns = one_hot.get_feature_names_out())
X_train_df

In [ ]:
X_train_df.info()

In [ ]:
X_test_df = pd.DataFrame(X_test_encoded, columns=one_hot.get_feature_names_out())
X_test_df

In [ ]:
X_test_df.info()

In [ ]:
X_train_corr = X_train_df.copy()
X_train_corr['churn'] = y_train.reset_index(drop=True)

correlaciones = X_train_corr.corr()['churn'].sort_values(ascending=False)
print(correlaciones)